### Objective of the NB

To practice clustering and classification algorithms with the datasets like amazon, WB indicators. 

Continue the study with NLP related analysis using the Quora answers

- I have to still understand, how to write out the model and load the model

- Need to understand how the feature selection works

- Practice setting up KPI based on the data Analysis and modeling

- Try to get hands on real world manufacturing data, and do the analysis

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
#always initiate the spark session with all the connectivity that is 
#available in the environment you are workin
sparkSQL = SparkSession.builder.appName("clustClass") \
                .config('spark.jars',"/usr/share/java/postgresql-42.2.26.jar") \
                .getOrCreate()

22/11/29 12:49:32 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
22/11/29 12:49:32 INFO SharedState: Warehouse path is 'file:/run/media/solverbot/repoA/gitFolders/dashBoard%20Designs/spark-warehouse'.
22/11/29 12:49:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
def writingCSVFiletoDatabase(session, csvFile,dbName,dbTableName):
    
    fileSparkDF = session.read.csv(csvFile,inferSchema=True,header=True)
    try:
        fileSparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')

In [5]:
#For writing the data to db, the spark session
def writingSparkDFtoDatabase(sparkDF,dbName,dbTableName):
    
    try:
        sparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')

In [6]:
sparkreader = sparkSQL.read
sparkctxt = sparkSQL.sparkContext

In [8]:
externalData = "externalData/"

In [12]:
#Reading in the amazonRA encoded data, that was prepared using the 
#amazon_delivery_analysis.ipynb

amazonEncoded = sparkreader.csv(externalData+"amazonRA_encoded.csv",
                               inferSchema=True, header=True)

22/11/29 13:08:21 INFO InMemoryFileIndex: It took 14 ms to list leaf files for 1 paths.
22/11/29 13:08:21 INFO InMemoryFileIndex: It took 3 ms to list leaf files for 1 paths.
22/11/29 13:08:21 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 13:08:21 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#73, None)) > 0)
22/11/29 13:08:21 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
22/11/29 13:08:21 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 350.4 KiB, free 364.8 MiB)
22/11/29 13:08:21 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 364.8 MiB)
22/11/29 13:08:21 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 172.17.0.1:38511 (size: 34.0 KiB, free: 366.2 MiB)
22/11/29 13:08:21 INFO SparkContext: Created broadcast 6 from csv at NativeMethodAccessorImpl.java:0
22/11/29 13:08:21 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes,

In [10]:
amazonEncoded.count()

22/11/29 13:06:07 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 13:06:07 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 13:06:07 INFO FileSourceStrategy: Output Data Schema: struct<>
22/11/29 13:06:07 INFO CodeGenerator: Code generated in 24.195704 ms
22/11/29 13:06:07 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 350.0 KiB, free 365.2 MiB)
22/11/29 13:06:07 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 365.2 MiB)
22/11/29 13:06:07 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 172.17.0.1:38511 (size: 34.0 KiB, free: 366.2 MiB)
22/11/29 13:06:07 INFO SparkContext: Created broadcast 3 from count at NativeMethodAccessorImpl.java:0
22/11/29 13:06:07 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
22/11/29 13:06:07 INFO DAGScheduler: Registering RDD 13 (count at NativeMethodAccessorImpl.java:0) 

10669

22/11/29 13:06:08 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 172.17.0.1:38511 in memory (size: 5.9 KiB, free: 366.2 MiB)
22/11/29 13:06:08 INFO BlockManagerInfo: Removed broadcast_4_piece0 on 172.17.0.1:38511 in memory (size: 8.3 KiB, free: 366.2 MiB)


In [13]:
amazonEncoded.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Delivery_person_ID: string (nullable = true)
 |-- Type_of_Vehicle: string (nullable = true)
 |-- DeliveryPersonAge: double (nullable = true)
 |-- DeliveryPersonRatings: double (nullable = true)
 |-- TypeOfVehicle: double (nullable = true)
 |-- DeliveryPersonID: double (nullable = true)
 |-- Restaurant_latitude: double (nullable = true)
 |-- Restaurant_longitude: double (nullable = true)
 |-- Delivery_location_latitude: double (nullable = true)
 |-- Delivery_location_longitude: double (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Weather_idx: double (nullable = true)
 |-- Road_traffic_density_idx: double (nullable = true)
 |-- Type_of_order_idx: double (nullable = true)
 |-- multiple_deliveries_idx: double (nullable = true)
 |-- IntervalPickup: integer (nullable = true)

22/11/29 13:18:22 INFO BlockManagerInfo: Removed broadcast_2_piece0 on 172.17.0.1:38511 in memory (size: 34.0 KiB, free: 366.2 MiB)
22/11/29 13:18:22 INFO B

Thinking of a way to create the clustering model around the amazon data. The road traffic density, restaurant location and delivery locations can be used in this case. 

In [14]:
amazonEncoded.select("Road_traffic_density_idx").distinct().show()

22/11/29 13:47:52 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 13:47:52 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 13:47:52 INFO FileSourceStrategy: Output Data Schema: struct<Road_traffic_density_idx: double>
22/11/29 13:47:52 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
22/11/29 13:47:52 INFO CodeGenerator: Code generated in 52.389035 ms
22/11/29 13:47:52 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 350.3 KiB, free 366.0 MiB)
22/11/29 13:47:52 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.9 MiB)
22/11/29 13:47:52 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.3 MiB)
22/11/29 13:47:52 INFO SparkContext: Created broadcast 10 from showString at NativeMethodAccessorImpl.java:0
22/11/29 13:47:52 INFO 

In [16]:
clusteringData = amazonEncoded.select(col("Road_traffic_density_idx").alias('label'),
                                     "Restaurant_latitude","Restaurant_longitude",
                                     "Delivery_location_longitude","Delivery_location_latitude",
                                     "Weather_idx","Type_of_order_idx")
clusteringData.show(2)

22/11/29 13:51:03 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 13:51:03 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 13:51:03 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 13:51:03 INFO CodeGenerator: Code generated in 30.931846 ms
22/11/29 13:51:03 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 350.3 KiB, free 366.0 MiB)
22/11/29 13:51:03 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.9 MiB)
22/11/29 13:51:03 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.3 MiB)
22/11/29 13:51:03 INFO SparkContext: Created broadcast 13 from showString at NativeMethodAccessorImpl.java:0
22/11/29 13:51:03 INFO FileSourceScanExec: Planning scan with bin packin

In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

### Opportunity to do both Classification and Clustering is opening up

The traffic density point would be linked with busy part of the cities, so that can be inferred by clustering and finding the centers

The traffic can be predicted based on the data that is available, and the delivery time can be estimated. The data can be used in different ways 

In [44]:
clusterAssy = VectorAssembler(inputCols=["Type_of_order_idx","Weather_idx",
                                        "Delivery_location_latitude","Delivery_location_longitude",
                                        "Restaurant_longitude","Restaurant_latitude"],
                             outputCol="features")
kmeansCluster = KMeans(featuresCol="features",k=5)
clusterPipe = Pipeline(stages=[clusterAssy,kmeansCluster])

In [45]:
(train,test) = amazonEncoded.randomSplit([0.7,0.3])

In [46]:
modelTrain = clusterPipe.fit(train)

22/11/29 14:14:50 INFO Instrumentation: [42aef167] Stage class: KMeans
22/11/29 14:14:50 INFO Instrumentation: [42aef167] Stage uid: KMeans_ce8a75624c79
22/11/29 14:14:50 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:14:50 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:14:50 INFO FileSourceStrategy: Output Data Schema: struct<ID: string, Delivery_person_ID: string, Type_of_Vehicle: string, DeliveryPersonAge: double, DeliveryPersonRatings: double ... 15 more fields>
22/11/29 14:14:50 INFO CodeGenerator: Code generated in 34.01239 ms
22/11/29 14:14:50 INFO MemoryStore: Block broadcast_51 stored as values in memory (estimated size 350.3 KiB, free 364.7 MiB)
22/11/29 14:14:50 INFO MemoryStore: Block broadcast_51_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.7 MiB)
22/11/29 14:14:50 INFO BlockManagerInfo: Added broadcast_51_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.1 MiB)
22/11/29 14:14:50 INFO SparkContext: Created broadcast 51 

22/11/29 14:14:51 INFO MemoryStore: Block rdd_112_0 stored as values in memory (estimated size 838.7 KiB, free 363.9 MiB)
22/11/29 14:14:51 INFO BlockManagerInfo: Added rdd_112_0 in memory on 172.17.0.1:38511 (size: 838.7 KiB, free: 365.3 MiB)
22/11/29 14:14:51 INFO Executor: Finished task 0.0 in stage 31.0 (TID 28). 1892 bytes result sent to driver
22/11/29 14:14:51 INFO TaskSetManager: Finished task 0.0 in stage 31.0 (TID 28) in 552 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:14:51 INFO TaskSchedulerImpl: Removed TaskSet 31.0, whose tasks have all completed, from pool 
22/11/29 14:14:51 INFO DAGScheduler: ResultStage 31 (takeSample at KMeans.scala:384) finished in 0.565 s
22/11/29 14:14:51 INFO DAGScheduler: Job 24 is finished. Cancelling potential speculative or zombie tasks for this job
22/11/29 14:14:51 INFO TaskSchedulerImpl: Killing all running tasks in stage 31: Stage finished
22/11/29 14:14:51 INFO DAGScheduler: Job 24 finished: takeSample at KMeans.scala:384, took 0.

22/11/29 14:14:51 INFO BlockManagerInfo: Removed broadcast_58_piece0 on 172.17.0.1:38511 in memory (size: 597.0 B, free: 365.2 MiB)
22/11/29 14:14:51 INFO MemoryStore: Block broadcast_61 stored as values in memory (estimated size 3.2 KiB, free 363.5 MiB)
22/11/29 14:14:51 INFO BlockManagerInfo: Removed broadcast_55_piece0 on 172.17.0.1:38511 in memory (size: 316.0 B, free: 365.2 MiB)
22/11/29 14:14:51 INFO MemoryStore: Block broadcast_61_piece0 stored as bytes in memory (estimated size 1574.0 B, free 363.5 MiB)
22/11/29 14:14:51 INFO BlockManagerInfo: Added broadcast_61_piece0 in memory on 172.17.0.1:38511 (size: 1574.0 B, free: 365.2 MiB)
22/11/29 14:14:51 INFO SparkContext: Created broadcast 61 from broadcast at KMeans.scala:429
22/11/29 14:14:51 INFO SparkContext: Starting job: countByValue at KMeans.scala:432
22/11/29 14:14:51 INFO DAGScheduler: Registering RDD 125 (countByValue at KMeans.scala:432) as input to shuffle 7
22/11/29 14:14:51 INFO DAGScheduler: Got job 30 (countByValue

22/11/29 14:14:52 INFO BlockManagerInfo: Removed broadcast_67_piece0 on 172.17.0.1:38511 in memory (size: 2.8 KiB, free: 365.1 MiB)
22/11/29 14:14:52 INFO SparkContext: Starting job: collectAsMap at KMeans.scala:315
22/11/29 14:14:52 INFO DAGScheduler: Registering RDD 131 (mapPartitions at KMeans.scala:294) as input to shuffle 10
22/11/29 14:14:52 INFO DAGScheduler: Got job 33 (collectAsMap at KMeans.scala:315) with 1 output partitions
22/11/29 14:14:52 INFO DAGScheduler: Final stage: ResultStage 44 (collectAsMap at KMeans.scala:315)
22/11/29 14:14:52 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 43)
22/11/29 14:14:52 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 43)
22/11/29 14:14:52 INFO DAGScheduler: Submitting ShuffleMapStage 43 (MapPartitionsRDD[131] at mapPartitions at KMeans.scala:294), which has no missing parents
22/11/29 14:14:52 INFO BlockManagerInfo: Removed broadcast_62_piece0 on 172.17.0.1:38511 in memory (size: 23.9 KiB, free: 365.2 MiB)
22/1

22/11/29 14:14:52 INFO BlockManagerInfo: Removed broadcast_60_piece0 on 172.17.0.1:38511 in memory (size: 23.4 KiB, free: 366.1 MiB)
22/11/29 14:14:52 INFO BlockManager: Removing RDD 116
22/11/29 14:14:52 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:14:52 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:14:52 INFO FileSourceStrategy: Output Data Schema: struct<ID: string, Delivery_person_ID: string, Type_of_Vehicle: string, DeliveryPersonAge: double, DeliveryPersonRatings: double ... 15 more fields>
22/11/29 14:14:52 INFO CodeGenerator: Code generated in 49.718374 ms
22/11/29 14:14:52 INFO MemoryStore: Block broadcast_80 stored as values in memory (estimated size 350.3 KiB, free 364.3 MiB)
22/11/29 14:14:52 INFO MemoryStore: Block broadcast_80_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.3 MiB)
22/11/29 14:14:52 INFO BlockManagerInfo: Added broadcast_80_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.1 MiB)
22/11/29 14:14:52 INFO 

In [56]:
modelTrain.transform(test).select("prediction","features").show(2)

22/11/29 14:18:30 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:18:30 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:18:30 INFO FileSourceStrategy: Output Data Schema: struct<ID: string, Delivery_person_ID: string, Type_of_Vehicle: string, DeliveryPersonAge: double, DeliveryPersonRatings: double ... 15 more fields>
22/11/29 14:18:30 INFO CodeGenerator: Code generated in 37.578967 ms
22/11/29 14:18:30 INFO MemoryStore: Block broadcast_85 stored as values in memory (estimated size 350.3 KiB, free 366.0 MiB)
22/11/29 14:18:30 INFO MemoryStore: Block broadcast_85_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.9 MiB)
22/11/29 14:18:30 INFO BlockManagerInfo: Added broadcast_85_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.3 MiB)
22/11/29 14:18:30 INFO SparkContext: Created broadcast 85 from showString at NativeMethodAccessorImpl.java:0
22/11/29 14:18:30 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, o

In [59]:
kmeans = KMeans(featuresCol="features",k=5)
kmeans.fit(clusterAssy.transform(train)).clusterCenters()

22/11/29 14:25:52 INFO Instrumentation: [9bf526c7] Stage class: KMeans
22/11/29 14:25:52 INFO Instrumentation: [9bf526c7] Stage uid: KMeans_4ab97b2a85d4
22/11/29 14:25:52 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:25:52 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:25:52 INFO FileSourceStrategy: Output Data Schema: struct<ID: string, Delivery_person_ID: string, Type_of_Vehicle: string, DeliveryPersonAge: double, DeliveryPersonRatings: double ... 15 more fields>
22/11/29 14:25:52 INFO MemoryStore: Block broadcast_87 stored as values in memory (estimated size 350.3 KiB, free 365.5 MiB)
22/11/29 14:25:52 INFO MemoryStore: Block broadcast_87_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.5 MiB)
22/11/29 14:25:52 INFO BlockManagerInfo: Added broadcast_87_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.2 MiB)
22/11/29 14:25:52 INFO SparkContext: Created broadcast 87 from rdd at Instrumentation.scala:62
22/11/29 14:25:52 INFO FileSour

22/11/29 14:25:53 INFO MemoryStore: Block broadcast_93 stored as values in memory (estimated size 54.8 KiB, free 364.4 MiB)
22/11/29 14:25:53 INFO MemoryStore: Block broadcast_93_piece0 stored as bytes in memory (estimated size 23.3 KiB, free 364.4 MiB)
22/11/29 14:25:53 INFO BlockManagerInfo: Added broadcast_93_piece0 in memory on 172.17.0.1:38511 (size: 23.3 KiB, free: 365.3 MiB)
22/11/29 14:25:53 INFO SparkContext: Created broadcast 93 from broadcast at DAGScheduler.scala:1513
22/11/29 14:25:53 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 55 (MapPartitionsRDD[171] at mapPartitionsWithIndex at KMeans.scala:409) (first 15 tasks are for partitions Vector(0))
22/11/29 14:25:53 INFO TaskSchedulerImpl: Adding task set 55.0 with 1 tasks resource profile 0
22/11/29 14:25:53 INFO TaskSetManager: Starting task 0.0 in stage 55.0 (TID 51) (172.17.0.1, executor driver, partition 0, PROCESS_LOCAL, 5353 bytes) taskResourceAssignments Map()
22/11/29 14:25:53 INFO Executor: Running

22/11/29 14:25:53 INFO Executor: Finished task 0.0 in stage 58.0 (TID 54). 2090 bytes result sent to driver
22/11/29 14:25:53 INFO TaskSetManager: Finished task 0.0 in stage 58.0 (TID 54) in 108 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:25:53 INFO TaskSchedulerImpl: Removed TaskSet 58.0, whose tasks have all completed, from pool 
22/11/29 14:25:53 INFO DAGScheduler: ShuffleMapStage 58 (countByValue at KMeans.scala:432) finished in 0.117 s
22/11/29 14:25:53 INFO DAGScheduler: looking for newly runnable stages
22/11/29 14:25:53 INFO DAGScheduler: running: Set()
22/11/29 14:25:53 INFO DAGScheduler: waiting: Set(ResultStage 59)
22/11/29 14:25:53 INFO DAGScheduler: failed: Set()
22/11/29 14:25:53 INFO DAGScheduler: Submitting ResultStage 59 (ShuffledRDD[179] at countByValue at KMeans.scala:432), which has no missing parents
22/11/29 14:25:53 INFO MemoryStore: Block broadcast_99 stored as values in memory (estimated size 5.2 KiB, free 364.2 MiB)
22/11/29 14:25:53 INFO MemoryStore:

22/11/29 14:25:53 INFO MemoryStore: Block broadcast_106_piece0 stored as bytes in memory (estimated size 23.7 KiB, free 364.0 MiB)
22/11/29 14:25:53 INFO BlockManagerInfo: Added broadcast_106_piece0 in memory on 172.17.0.1:38511 (size: 23.7 KiB, free: 365.2 MiB)
22/11/29 14:25:53 INFO SparkContext: Created broadcast 106 from broadcast at DAGScheduler.scala:1513
22/11/29 14:25:53 INFO DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 62 (MapPartitionsRDD[182] at mapPartitions at KMeans.scala:294) (first 15 tasks are for partitions Vector(0))
22/11/29 14:25:53 INFO TaskSchedulerImpl: Adding task set 62.0 with 1 tasks resource profile 0
22/11/29 14:25:53 INFO TaskSetManager: Starting task 0.0 in stage 62.0 (TID 58) (172.17.0.1, executor driver, partition 0, PROCESS_LOCAL, 5278 bytes) taskResourceAssignments Map()
22/11/29 14:25:53 INFO Executor: Running task 0.0 in stage 62.0 (TID 58)
22/11/29 14:25:53 INFO BlockManager: Found block rdd_165_0 locally
22/11/29 14:25:53 INFO Exe

22/11/29 14:25:53 INFO BlockManager: Found block rdd_165_0 locally
22/11/29 14:25:53 INFO Executor: Finished task 0.0 in stage 66.0 (TID 62). 2203 bytes result sent to driver
22/11/29 14:25:53 INFO TaskSetManager: Finished task 0.0 in stage 66.0 (TID 62) in 29 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:25:53 INFO TaskSchedulerImpl: Removed TaskSet 66.0, whose tasks have all completed, from pool 
22/11/29 14:25:53 INFO DAGScheduler: ShuffleMapStage 66 (mapPartitions at KMeans.scala:294) finished in 0.051 s
22/11/29 14:25:53 INFO DAGScheduler: looking for newly runnable stages
22/11/29 14:25:53 INFO DAGScheduler: running: Set()
22/11/29 14:25:53 INFO DAGScheduler: waiting: Set(ResultStage 67)
22/11/29 14:25:53 INFO DAGScheduler: failed: Set()
22/11/29 14:25:53 INFO DAGScheduler: Submitting ResultStage 67 (ShuffledRDD[187] at reduceByKey at KMeans.scala:315), which has no missing parents
22/11/29 14:25:53 INFO MemoryStore: Block broadcast_115 stored as values in memory (estimate

22/11/29 14:25:54 INFO SparkContext: Starting job: collect at ClusteringSummary.scala:49
22/11/29 14:25:54 INFO DAGScheduler: Got job 51 (collect at ClusteringSummary.scala:49) with 1 output partitions
22/11/29 14:25:54 INFO DAGScheduler: Final stage: ResultStage 70 (collect at ClusteringSummary.scala:49)
22/11/29 14:25:54 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 69)
22/11/29 14:25:54 INFO DAGScheduler: Missing parents: List()
22/11/29 14:25:54 INFO DAGScheduler: Submitting ResultStage 70 (MapPartitionsRDD[194] at collect at ClusteringSummary.scala:49), which has no missing parents
22/11/29 14:25:54 INFO MemoryStore: Block broadcast_118 stored as values in memory (estimated size 58.9 KiB, free 364.9 MiB)
22/11/29 14:25:54 INFO MemoryStore: Block broadcast_118_piece0 stored as bytes in memory (estimated size 27.0 KiB, free 364.9 MiB)
22/11/29 14:25:54 INFO BlockManagerInfo: Added broadcast_118_piece0 in memory on 172.17.0.1:38511 (size: 27.0 KiB, free: 366.1 MiB)


[array([ 1.48093842,  2.4115347 , 15.239162  , 79.40888819, 79.34755876,
        15.17783257]),
 array([1.52622061, 2.34538879, 0.06019892, 0.06019892, 0.        ,
        0.        ]),
 array([ 1.4517491 ,  2.49939686, 25.98590155, 79.93074181, 79.86386605,
        25.91902579]),
 array([ 1.47923681,  2.43995511, 12.34483727, 76.93604159, 76.87223126,
        11.68501101]),
 array([ 1.47013721,  2.46004843, 20.6528718 , 73.90865709, 73.84663126,
        20.59084598])]

In [61]:
train.show(2)

22/11/29 14:30:41 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:30:41 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:30:41 INFO FileSourceStrategy: Output Data Schema: struct<ID: string, Delivery_person_ID: string, Type_of_Vehicle: string, DeliveryPersonAge: double, DeliveryPersonRatings: double ... 15 more fields>
22/11/29 14:30:41 INFO CodeGenerator: Code generated in 24.595884 ms
22/11/29 14:30:41 INFO MemoryStore: Block broadcast_151 stored as values in memory (estimated size 350.3 KiB, free 363.5 MiB)
22/11/29 14:30:41 INFO MemoryStore: Block broadcast_151_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 363.4 MiB)
22/11/29 14:30:41 INFO BlockManagerInfo: Added broadcast_151_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.0 MiB)
22/11/29 14:30:41 INFO SparkContext: Created broadcast 151 from showString at NativeMethodAccessorImpl.java:0
22/11/29 14:30:41 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

In [64]:
assembler =clusterAssy = VectorAssembler(inputCols=["Type_of_order_idx","Weather_idx",
                                        "Delivery_location_latitude","Delivery_location_longitude",
                                        "Restaurant_longitude","Restaurant_latitude"],
                             outputCol="features")

log_reg = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
pipeline = Pipeline(stages = [assembler, log_reg])

In [75]:
randomClass = RandomForestClassifier()
ranPipeline = Pipeline(stages=[assembler,randomClass])

In [66]:
(train_c, test_c) = clusteringData.randomSplit([0.8,0.2])

In [67]:
modelPipe = pipeline.fit(train_c)

22/11/29 14:33:16 INFO Instrumentation: [38942545] Stage class: LogisticRegression
22/11/29 14:33:16 INFO Instrumentation: [38942545] Stage uid: LogisticRegression_407bb945bcc2
22/11/29 14:33:16 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:33:16 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:33:16 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:33:16 INFO CodeGenerator: Code generated in 28.340644 ms
22/11/29 14:33:16 INFO MemoryStore: Block broadcast_186 stored as values in memory (estimated size 350.3 KiB, free 362.8 MiB)
22/11/29 14:33:16 INFO MemoryStore: Block broadcast_186_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 362.8 MiB)
22/11/29 14:33:16 INFO BlockManagerInfo: Added broadcast_186_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 365.9 MiB)
2

22/11/29 14:33:17 INFO MemoryStore: Block rdd_301_0 stored as values in memory (estimated size 467.7 KiB, free 361.8 MiB)
22/11/29 14:33:17 INFO BlockManagerInfo: Added rdd_301_0 in memory on 172.17.0.1:38511 (size: 467.7 KiB, free: 365.4 MiB)
22/11/29 14:33:17 INFO Executor: Finished task 0.0 in stage 106.0 (TID 100). 3093 bytes result sent to driver
22/11/29 14:33:17 INFO TaskSetManager: Finished task 0.0 in stage 106.0 (TID 100) in 236 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:33:17 INFO TaskSchedulerImpl: Removed TaskSet 106.0, whose tasks have all completed, from pool 
22/11/29 14:33:17 INFO DAGScheduler: ResultStage 106 (treeAggregate at RDDLossFunction.scala:61) finished in 0.248 s
22/11/29 14:33:17 INFO DAGScheduler: Job 81 is finished. Cancelling potential speculative or zombie tasks for this job
22/11/29 14:33:17 INFO TaskSchedulerImpl: Killing all running tasks in stage 106: Stage finished
22/11/29 14:33:17 INFO DAGScheduler: Job 81 finished: treeAggregate at RDDL

22/11/29 14:33:17 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 109 (MapPartitionsRDD[305] at treeAggregate at RDDLossFunction.scala:61) (first 15 tasks are for partitions Vector(0))
22/11/29 14:33:17 INFO TaskSchedulerImpl: Adding task set 109.0 with 1 tasks resource profile 0
22/11/29 14:33:17 INFO TaskSetManager: Starting task 0.0 in stage 109.0 (TID 103) (172.17.0.1, executor driver, partition 0, PROCESS_LOCAL, 5026 bytes) taskResourceAssignments Map()
22/11/29 14:33:17 INFO Executor: Running task 0.0 in stage 109.0 (TID 103)
22/11/29 14:33:17 INFO BlockManager: Found block rdd_301_0 locally
22/11/29 14:33:17 INFO Executor: Finished task 0.0 in stage 109.0 (TID 103). 3050 bytes result sent to driver
22/11/29 14:33:17 INFO TaskSetManager: Finished task 0.0 in stage 109.0 (TID 103) in 34 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:33:17 INFO TaskSchedulerImpl: Removed TaskSet 109.0, whose tasks have all completed, from pool 
22/11/29 14:33:17 INFO DAGSchedul

22/11/29 14:33:18 INFO Executor: Running task 0.0 in stage 112.0 (TID 106)
22/11/29 14:33:18 INFO BlockManager: Found block rdd_301_0 locally
22/11/29 14:33:18 INFO Executor: Finished task 0.0 in stage 112.0 (TID 106). 3093 bytes result sent to driver
22/11/29 14:33:18 INFO TaskSetManager: Finished task 0.0 in stage 112.0 (TID 106) in 17 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:33:18 INFO TaskSchedulerImpl: Removed TaskSet 112.0, whose tasks have all completed, from pool 
22/11/29 14:33:18 INFO DAGScheduler: ResultStage 112 (treeAggregate at RDDLossFunction.scala:61) finished in 0.022 s
22/11/29 14:33:18 INFO DAGScheduler: Job 87 is finished. Cancelling potential speculative or zombie tasks for this job
22/11/29 14:33:18 INFO TaskSchedulerImpl: Killing all running tasks in stage 112: Stage finished
22/11/29 14:33:18 INFO DAGScheduler: Job 87 finished: treeAggregate at RDDLossFunction.scala:61, took 0.026092 s
22/11/29 14:33:18 INFO TorrentBroadcast: Destroying Broadcast(203

22/11/29 14:33:18 INFO MemoryStore: Block broadcast_214 stored as values in memory (estimated size 56.1 KiB, free 361.3 MiB)
22/11/29 14:33:18 INFO MemoryStore: Block broadcast_214_piece0 stored as bytes in memory (estimated size 23.6 KiB, free 361.3 MiB)
22/11/29 14:33:18 INFO BlockManagerInfo: Added broadcast_214_piece0 in memory on 172.17.0.1:38511 (size: 23.6 KiB, free: 365.3 MiB)
22/11/29 14:33:18 INFO SparkContext: Created broadcast 214 from broadcast at DAGScheduler.scala:1513
22/11/29 14:33:18 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 117 (MapPartitionsRDD[313] at treeAggregate at RDDLossFunction.scala:61) (first 15 tasks are for partitions Vector(0))
22/11/29 14:33:18 INFO TaskSchedulerImpl: Adding task set 117.0 with 1 tasks resource profile 0
22/11/29 14:33:18 INFO TaskSetManager: Starting task 0.0 in stage 117.0 (TID 111) (172.17.0.1, executor driver, partition 0, PROCESS_LOCAL, 5026 bytes) taskResourceAssignments Map()
22/11/29 14:33:18 INFO Executor: 

In [69]:
prediction = modelPipe.transform(test_c)

In [72]:
prediction.select("prediction","label").show()

22/11/29 14:34:40 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:34:40 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:34:40 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:34:40 INFO MemoryStore: Block broadcast_222 stored as values in memory (estimated size 350.3 KiB, free 361.9 MiB)
22/11/29 14:34:40 INFO MemoryStore: Block broadcast_222_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 361.9 MiB)
22/11/29 14:34:40 INFO BlockManagerInfo: Added broadcast_222_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 365.9 MiB)
22/11/29 14:34:40 INFO SparkContext: Created broadcast 222 from showString at NativeMethodAccessorImpl.java:0
22/11/29 14:34:40 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4

In [73]:
from pyspark.ml.evaluation import ClusteringEvaluator,MulticlassClassificationEvaluator

In [74]:
evaluation = MulticlassClassificationEvaluator(labelCol='label',predictionCol="prediction")
evaluation.evaluate(prediction.select("label","prediction"))

22/11/29 14:39:51 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:39:51 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:39:51 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:39:51 INFO CodeGenerator: Code generated in 26.120088 ms
22/11/29 14:39:51 INFO MemoryStore: Block broadcast_224 stored as values in memory (estimated size 350.3 KiB, free 361.5 MiB)
22/11/29 14:39:51 INFO MemoryStore: Block broadcast_224_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 361.5 MiB)
22/11/29 14:39:51 INFO BlockManagerInfo: Added broadcast_224_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 365.8 MiB)
22/11/29 14:39:51 INFO SparkContext: Created broadcast 224 from rdd at MulticlassClassificationEvaluator.scala:203
22/11/29 14:39:51 INFO FileSourceScanExec: Planning scan with b

0.18942382496005106

In [76]:
randModel = ranPipeline.fit(train_c)
prediction_rand  = randModel.transform(test_c)

22/11/29 14:48:03 INFO Instrumentation: [e7d237fe] Stage class: RandomForestClassifier
22/11/29 14:48:03 INFO Instrumentation: [e7d237fe] Stage uid: RandomForestClassifier_93c04c4e6474
22/11/29 14:48:03 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:48:03 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:48:03 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:48:03 INFO MemoryStore: Block broadcast_227 stored as values in memory (estimated size 350.3 KiB, free 361.0 MiB)
22/11/29 14:48:03 INFO MemoryStore: Block broadcast_227_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 361.0 MiB)
22/11/29 14:48:03 INFO BlockManagerInfo: Added broadcast_227_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 365.7 MiB)
22/11/29 14:48:03 INFO SparkContext: Created broadcast 227 fro

22/11/29 14:48:04 INFO FileScanRDD: Reading File path: file:///run/media/solverbot/repoA/gitFolders/dashBoard%20Designs/externalData/amazonRA_encoded.csv/part-00000-6df05801-2b9a-4d6c-9a42-0d8304d1f37f-c000.csv, range: 0-1273099, partition values: [empty row]
22/11/29 14:48:04 INFO Executor: Finished task 0.0 in stage 127.0 (TID 120). 1798 bytes result sent to driver
22/11/29 14:48:04 INFO TaskSetManager: Finished task 0.0 in stage 127.0 (TID 120) in 148 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:48:04 INFO TaskSchedulerImpl: Removed TaskSet 127.0, whose tasks have all completed, from pool 
22/11/29 14:48:04 INFO DAGScheduler: ResultStage 127 (take at DecisionTreeMetadata.scala:119) finished in 0.155 s
22/11/29 14:48:04 INFO DAGScheduler: Job 100 is finished. Cancelling potential speculative or zombie tasks for this job
22/11/29 14:48:04 INFO TaskSchedulerImpl: Killing all running tasks in stage 127: Stage finished
22/11/29 14:48:04 INFO DAGScheduler: Job 100 finished: take a

22/11/29 14:48:05 INFO SparkContext: Starting job: collectAsMap at RandomForest.scala:663
22/11/29 14:48:05 INFO DAGScheduler: Registering RDD 372 (mapPartitions at RandomForest.scala:644) as input to shuffle 28
22/11/29 14:48:05 INFO DAGScheduler: Got job 103 (collectAsMap at RandomForest.scala:663) with 1 output partitions
22/11/29 14:48:05 INFO DAGScheduler: Final stage: ResultStage 132 (collectAsMap at RandomForest.scala:663)
22/11/29 14:48:05 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 131)
22/11/29 14:48:05 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 131)
22/11/29 14:48:05 INFO DAGScheduler: Submitting ShuffleMapStage 131 (MapPartitionsRDD[372] at mapPartitions at RandomForest.scala:644), which has no missing parents
22/11/29 14:48:05 INFO MemoryStore: Block broadcast_238 stored as values in memory (estimated size 60.6 KiB, free 361.7 MiB)
22/11/29 14:48:05 INFO MemoryStore: Block broadcast_238_piece0 stored as bytes in memory (estimated size 26.1

22/11/29 14:48:06 INFO Executor: Finished task 0.0 in stage 132.0 (TID 125). 6833 bytes result sent to driver
22/11/29 14:48:06 INFO TaskSetManager: Finished task 0.0 in stage 132.0 (TID 125) in 321 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:48:06 INFO TaskSchedulerImpl: Removed TaskSet 132.0, whose tasks have all completed, from pool 
22/11/29 14:48:06 INFO DAGScheduler: ResultStage 132 (collectAsMap at RandomForest.scala:663) finished in 0.329 s
22/11/29 14:48:06 INFO DAGScheduler: Job 103 is finished. Cancelling potential speculative or zombie tasks for this job
22/11/29 14:48:06 INFO TaskSchedulerImpl: Killing all running tasks in stage 132: Stage finished
22/11/29 14:48:06 INFO DAGScheduler: Job 103 finished: collectAsMap at RandomForest.scala:663, took 0.903964 s
22/11/29 14:48:06 INFO TorrentBroadcast: Destroying Broadcast(237) (from destroy at RandomForest.scala:674)
22/11/29 14:48:06 INFO BlockManagerInfo: Removed broadcast_237_piece0 on 172.17.0.1:38511 in memory (s

22/11/29 14:48:06 INFO Executor: Finished task 0.0 in stage 133.0 (TID 126). 2090 bytes result sent to driver
22/11/29 14:48:06 INFO TaskSetManager: Finished task 0.0 in stage 133.0 (TID 126) in 391 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:48:06 INFO TaskSchedulerImpl: Removed TaskSet 133.0, whose tasks have all completed, from pool 
22/11/29 14:48:06 INFO DAGScheduler: ShuffleMapStage 133 (mapPartitions at RandomForest.scala:644) finished in 0.397 s
22/11/29 14:48:06 INFO DAGScheduler: looking for newly runnable stages
22/11/29 14:48:06 INFO DAGScheduler: running: Set()
22/11/29 14:48:06 INFO DAGScheduler: waiting: Set(ResultStage 134)
22/11/29 14:48:06 INFO DAGScheduler: failed: Set()
22/11/29 14:48:06 INFO DAGScheduler: Submitting ResultStage 134 (MapPartitionsRDD[377] at map at RandomForest.scala:663), which has no missing parents
22/11/29 14:48:06 INFO MemoryStore: Block broadcast_242 stored as values in memory (estimated size 11.1 KiB, free 360.5 MiB)
22/11/29 14:48:0

22/11/29 14:48:07 INFO Executor: Finished task 0.0 in stage 137.0 (TID 130). 2090 bytes result sent to driver
22/11/29 14:48:07 INFO TaskSetManager: Finished task 0.0 in stage 137.0 (TID 130) in 131 ms on 172.17.0.1 (executor driver) (1/1)
22/11/29 14:48:07 INFO TaskSchedulerImpl: Removed TaskSet 137.0, whose tasks have all completed, from pool 
22/11/29 14:48:07 INFO DAGScheduler: ShuffleMapStage 137 (mapPartitions at RandomForest.scala:644) finished in 0.141 s
22/11/29 14:48:07 INFO DAGScheduler: looking for newly runnable stages
22/11/29 14:48:07 INFO DAGScheduler: running: Set()
22/11/29 14:48:07 INFO DAGScheduler: waiting: Set(ResultStage 138)
22/11/29 14:48:07 INFO DAGScheduler: failed: Set()
22/11/29 14:48:07 INFO DAGScheduler: Submitting ResultStage 138 (MapPartitionsRDD[383] at map at RandomForest.scala:663), which has no missing parents
22/11/29 14:48:07 INFO MemoryStore: Block broadcast_248 stored as values in memory (estimated size 17.6 KiB, free 360.2 MiB)
22/11/29 14:48:0

22/11/29 14:48:08 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:48:08 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:48:08 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:48:08 INFO MemoryStore: Block broadcast_252 stored as values in memory (estimated size 350.3 KiB, free 362.0 MiB)
22/11/29 14:48:08 INFO MemoryStore: Block broadcast_252_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 362.0 MiB)
22/11/29 14:48:08 INFO BlockManagerInfo: Added broadcast_252_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 365.9 MiB)
22/11/29 14:48:08 INFO SparkContext: Created broadcast 252 from rdd at ClassificationSummary.scala:58
22/11/29 14:48:08 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 b

In [77]:
prediction_rand.select("label","prediction").show(2)

22/11/29 14:48:28 INFO FileSourceStrategy: Pushed Filters: 
22/11/29 14:48:28 INFO FileSourceStrategy: Post-Scan Filters: 
22/11/29 14:48:28 INFO FileSourceStrategy: Output Data Schema: struct<Restaurant_latitude: double, Restaurant_longitude: double, Delivery_location_latitude: double, Delivery_location_longitude: double, Weather_idx: double ... 5 more fields>
22/11/29 14:48:28 INFO CodeGenerator: Code generated in 35.226193 ms
22/11/29 14:48:28 INFO MemoryStore: Block broadcast_253 stored as values in memory (estimated size 350.3 KiB, free 364.8 MiB)
22/11/29 14:48:28 INFO MemoryStore: Block broadcast_253_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.8 MiB)
22/11/29 14:48:28 INFO BlockManagerInfo: Added broadcast_253_piece0 in memory on 172.17.0.1:38511 (size: 34.1 KiB, free: 366.2 MiB)
22/11/29 14:48:28 INFO SparkContext: Created broadcast 253 from showString at NativeMethodAccessorImpl.java:0
22/11/29 14:48:28 INFO FileSourceScanExec: Planning scan with bin pa